# Visualize pymoo result
Load the pymoo result object saved with pickle and visualize it.

ONOSControllerPlacement needs to be definded to load the result object.
So, execute the following cell first.

In [1]:
#!/usr/bin/env python
import numpy as np
import networkx as nx
import math
import pickle
from pymoo.core.problem import ElementwiseProblem

class ONOSControllerPlacement(ElementwiseProblem):
    def __init__(self, num_nodes, distance_matrix, shortest_paths, graph, **kwargs):
        super().__init__(n_var=2*num_nodes, 
                         n_obj=4, 
                         n_constr=2, 
                         xl=0, xu=1, 
                         **kwargs)
        self.num_nodes = num_nodes
        self.distance_matrix = distance_matrix
        self.shortest_paths = shortest_paths
        self.graph = graph
    
    def _evaluate(self, x, out, *args, **kwargs):
        controller_nodes = x[:self.num_nodes]   # first half is controller placement
        atomix_nodes = x[self.num_nodes:]       # second half is atomix placement


        num_controller = np.sum(controller_nodes)
        num_atomix = np.sum(atomix_nodes)

        # Obj1: Minimize number of contrtoller
        f1 = num_controller

        # Obj2: Minimize number of atomix
        f2 = num_atomix

        # Obj3: Minimize average FSP
        f3 = calculate_FST(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                           self.shortest_paths)
        
        f4 = calculate_BC(self.num_nodes, 
                           controller_nodes, 
                           atomix_nodes, 
                           self.distance_matrix, 
                        #    self.shortest_paths,
                           self.graph)

        # Constr1: The number of controller is equal to or greater than 2
        g1 = 2 - num_controller

        # Constr2: The number of atomix is equal to or greater than 3
        g2 = 3 - num_atomix
        
        # Add the centrality metrix into optimazing objectives:
        # 1. Nearest controller for each switch
        # 2. The number of controlled switches for each controller should be <= limit_num_switches_controlled (limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)))
        # 3. return value should be the variance for all controller's betweenness centrality
        out["F"] = [f1, f2, f3, f4]
        out["G"] = [g1, g2]


def calculate_FST(num_nodes, controller_nodes, atomix_nodes, distance_matrix, shortest_paths):
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()
    atomix_list = np.nonzero(atomix_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        for c in controller_list:
            if distance_matrix[s][c] < delay:
                delay = distance_matrix[s][c]
                nearest_controller = c
        controller_of.append(nearest_controller)    

    # calculate average delay to atomix nodes from each controller
    average_atomix_delay_from = {}
    for c in controller_list:
        delay = []
        for a in atomix_list:
            delay.append(distance_matrix[c][a])
        average_atomix_delay_from[c] = np.mean(delay)

    # find the nearest atomix for each atomix and calculate average delay
    atomix_atomix_delays = []
    for a1 in atomix_list:
        delay = math.inf
        for a2 in atomix_list:
            if(a1 == a2):
                continue
            if distance_matrix[a1][a2] < delay:
                delay = distance_matrix[a1][a2]
        atomix_atomix_delays.append(delay)
    average_atomix_atomix_delay = np.mean(atomix_atomix_delays)
    FTSs = []
    for source in range(num_nodes):
        for distination in range(num_nodes):
            if(source == distination):
                continue
            delay = 0
            is_controlled_by_single_controller = True
            counted_controllers = []
            for s in shortest_paths[source][distination]:
                # switch-controller delay
                delay += distance_matrix[s][controller_of[s]] * 4

                # controller-atomix delay
                if(s == source):
                    delay += average_atomix_delay_from[controller_of[s]] * 2
                elif(s != distination):
                    if(controller_of[s] != controller_of[source]):
                        is_controlled_by_single_controller = False
                        if(not controller_of[s] in counted_controllers):
                            counted_controllers.append(controller_of[s])
                            delay += average_atomix_delay_from[controller_of[s]]
                else:
                    if(controller_of[s] == controller_of[source]):
                        if(not is_controlled_by_single_controller):
                            delay += average_atomix_delay_from[controller_of[s]]
                    else:
                        delay += average_atomix_delay_from[controller_of[s]] * 2
            
            # atomix-atomix delay
            delay +=  average_atomix_atomix_delay * 2
            FTSs.append(delay)

    return np.mean(FTSs)



def calculate_BC(num_nodes, controller_nodes, atomix_nodes, distance_matrix, graph):
    G = nx.Graph()
    for node1 in range(len(graph)):
        G.add_node(str(node1))
        for node2, delay in graph[node1].items():
            G.add_edge(str(node1), str(node2), weight=delay)
    
    # The list of betweenness centrality for all switches
    nodes_bc=nx.current_flow_betweenness_centrality(G, normalized=True, weight=None, dtype='float', solver='full')
    num_controller = np.sum(controller_nodes)
    num_atomix = np.sum(atomix_nodes)
    controller_list = np.nonzero(controller_nodes)[0].tolist()

    if(num_controller == 0 or num_atomix ==0):
        return math.inf

    # find the nearest controller for each switch
    controller_of = []
    limit_num_switches_controlled=int(math.ceil(num_nodes/num_controller)) # balance the number of switches controllers can control 
    switches_bc_of_controller_ = dict.fromkeys((range(num_nodes)),0) # list of sum of betweenness centrality of switches for each controller
    for s in range(num_nodes):
        delay = math.inf
        nearest_controller = None
        controlled_switches=[]
        for c in controller_list:
            # Conditions: nearest controller (with the lowest delay) && the number of switches for each controller < limit_num_switches_controlled
            if distance_matrix[s][c] < delay and controller_of.count(c) < limit_num_switches_controlled:
                delay = distance_matrix[s][c]
                nearest_controller = c
                controlled_switches.append(s)
        switches_bc_of_controller_[nearest_controller] += nodes_bc[str(s)]
        controller_of.append(nearest_controller)
    
    # Simplify switches_bc_of_controller_ (only need value for calculating variance)
    bc_array = []
    for i in switches_bc_of_controller_.values():
        bc_array.append(i)

    # return variance value can show the degree of balance within all controllers
    return np.var(bc_array)

### Load kgbdmoea results for all topos

In [2]:
with open('res_bc_Cogent_kgbdmoea.pkl','rb') as f_Cogent:
    res_12_Cogent = pickle.load(f_Cogent)
with open('res_bc_UsCarrier_kgbdmoea.pkl','rb') as f_UsCarrier:
    res_12_UsCarrier = pickle.load(f_UsCarrier)
with open('res_bc_HiberniaGlobal_kgbdmoea.pkl','rb') as f_HiberniaGlobal:
    res_12_HiberniaGlobal = pickle.load(f_HiberniaGlobal)
with open('res_bc_Colt_kgbdmoea.pkl','rb') as f_Colt:
    res_12_Colt = pickle.load(f_Colt)
with open('res_bc_Funet_kgbdmoea.pkl','rb') as f_Funet:
    res_12_Funet = pickle.load(f_Funet)
with open('res_bc_Abvt_kgbdmoea.pkl','rb') as f_Abvt:
    res_12_Abvt = pickle.load(f_Abvt)
with open('res_bc_Intellifiber_kgbdmoea.pkl','rb') as f_Intellifiber:
    res_12_Intellifiber = pickle.load(f_Intellifiber)
with open('res_bc_TataNld_kgbdmoea.pkl','rb') as f_TataNld:
    res_12_TataNld = pickle.load(f_TataNld)
with open('res_bc_Internode_kgbdmoea.pkl','rb') as f_Internode:
    res_12_Internode = pickle.load(f_Internode)
with open('res_bc_Missouri_kgbdmoea.pkl','rb') as f_Missouri:
    res_12_Missouri = pickle.load(f_Missouri)
with open('res_bc_Ion_kgbdmoea.pkl','rb') as f_Ion:
    res_12_Ion = pickle.load(f_Ion)
with open('res_bc_Palmetto_kgbdmoea.pkl','rb') as f_Palmetto:
    res_12_Palmetto = pickle.load(f_Palmetto)

## Hypervolume
1. Store values

In [ ]:
# kgbdmoea
F12_Cogent=res_12_Cogent.F
F12_UsCarrier=res_12_UsCarrier.F
F12_HiberniaGlobal=res_12_HiberniaGlobal.F
F12_Colt=res_12_Colt.F
F12_Funet=res_12_Funet.F
F12_Abvt=res_12_Abvt.F
F12_Intellifiber=res_12_Intellifiber.F
F12_TataNld=res_12_TataNld.F
F12_Internode=res_12_Internode.F
F12_Missouri=res_12_Missouri.F
F12_Ion=res_12_Ion.F
F12_Palmetto=res_12_Palmetto.F

# # Nadir Point from 9 Algorithms: 
# ref_point = [1.81000000e+02 6.20000000e+01 3.92281875e+03 1.70537209e+00]
# # Nadir Point from 4 Algorithms: 
# ref_point = [160.  77.  inf  inf]
# # Nadir Point from all (13) Algorithms: 
# ref_point = [181.  77.  inf  inf]

ref_point = [181, 77, 3922.81875, 1.70537209]

hist_F12_Cogent = []
hist_F12_UsCarrier = []
hist_F12_HiberniaGlobal = []
hist_F12_Colt = []
hist_F12_Funet = []
hist_F12_Abvt = []
hist_F12_Intellifiber = []
hist_F12_TataNld = []
hist_F12_Internode = []
hist_F12_Missouri = []
hist_F12_Ion = []
hist_F12_Palmetto = []


for algo12_Cogent in res_12_Cogent.history:
    opt12_Cogent = algo12_Cogent.opt
    feas12_Cogent = np.where(opt12_Cogent.get("feasible"))[0]
    hist_F12_Cogent.append(opt12_Cogent.get("F")[feas12_Cogent])
for algo12_UsCarrier in res_12_UsCarrier.history:
    opt12_UsCarrier = algo12_UsCarrier.opt
    feas12_UsCarrier = np.where(opt12_UsCarrier.get("feasible"))[0]
    hist_F12_UsCarrier.append(opt12_UsCarrier.get("F")[feas12_UsCarrier])
for algo12_HiberniaGlobal in res_12_HiberniaGlobal.history:
    opt12_HiberniaGlobal = algo12_HiberniaGlobal.opt
    feas12_HiberniaGlobal = np.where(opt12_HiberniaGlobal.get("feasible"))[0]
    hist_F12_HiberniaGlobal.append(opt12_HiberniaGlobal.get("F")[feas12_HiberniaGlobal])
for algo12_Colt in res_12_Colt.history:
    opt12_Colt = algo12_Colt.opt
    feas12_Colt = np.where(opt12_Colt.get("feasible"))[0]
    hist_F12_Colt.append(opt12_Colt.get("F")[feas12_Colt])
for algo12_Funet in res_12_Funet.history:
    opt12_Funet = algo12_Funet.opt
    feas12_Funet = np.where(opt12_Funet.get("feasible"))[0]
    hist_F12_Funet.append(opt12_Funet.get("F")[feas12_Funet])
for algo12_Abvt in res_12_Abvt.history:
    opt12_Abvt = algo12_Abvt.opt
    feas12_Abvt = np.where(opt12_Abvt.get("feasible"))[0]
    hist_F12_Abvt.append(opt12_Abvt.get("F")[feas12_Abvt])
for algo12_Intellifiber in res_12_Intellifiber.history:
    opt12_Intellifiber = algo12_Intellifiber.opt
    feas12_Intellifiber = np.where(opt12_Intellifiber.get("feasible"))[0]
    hist_F12_Intellifiber.append(opt12_Intellifiber.get("F")[feas12_Intellifiber])
for algo12_TataNld in res_12_TataNld.history:
    opt12_TataNld = algo12_TataNld.opt
    feas12_TataNld = np.where(opt12_TataNld.get("feasible"))[0]
    hist_F12_TataNld.append(opt12_TataNld.get("F")[feas12_TataNld])
for algo12_Internode in res_12_Internode.history:
    opt12_Internode = algo12_Internode.opt
    feas12_Internode= np.where(opt12_Internode.get("feasible"))[0]
    hist_F12_Internode.append(opt12_Internode.get("F")[feas12_Internode])
for algo12_Missouri in res_12_Missouri.history:
    opt12_Missouri = algo12_Missouri.opt
    feas12_Missouri = np.where(opt12_Missouri.get("feasible"))[0]
    hist_F12_Missouri.append(opt12_Missouri.get("F")[feas12_Missouri])
for algo12_Ion in res_12_Ion.history:
    opt12_Ion = algo12_Ion.opt
    feas12_Ion = np.where(opt12_Ion.get("feasible"))[0]
    hist_F12_Ion.append(opt12_Ion.get("F")[feas12_Ion])
for algo12_Palmetto in res_12_Palmetto.history:
    opt12_Palmetto = algo12_Palmetto.opt
    feas12_Palmetto = np.where(opt12_Palmetto.get("feasible"))[0]
    hist_F12_Palmetto.append(opt12_Palmetto.get("F")[feas12_Palmetto])

2. Calculate hypervolume and merge them (Mean value) for kgbdmoea algorithm

In [ ]:
from pymoo.indicators.hv import Hypervolume

metric = Hypervolume(ref_point= np.array(ref_point))

hv_12_Cogent = [metric.do(_F12_Cogent) for _F12_Cogent in hist_F12_Cogent]
hv_12_UsCarrier = [metric.do(_F12_UsCarrier) for _F12_UsCarrier in hist_F12_UsCarrier]
hv_12_HiberniaGlobal = [metric.do(_F12_HiberniaGlobal) for _F12_HiberniaGlobal in hist_F12_HiberniaGlobal]
hv_12_Colt = [metric.do(_F12_Colt) for _F12_Colt in hist_F12_Colt]
hv_12_Funet = [metric.do(_F12_Funet) for _F12_Funet in hist_F12_Funet]
hv_12_Abvt = [metric.do(_F12_Abvt) for _F12_Abvt in hist_F12_Abvt]
hv_12_Intellifiber = [metric.do(_F12_Intellifiber) for _F12_Intellifiber in hist_F12_Intellifiber]
hv_12_TataNld = [metric.do(_F12_TataNld) for _F12_TataNld in hist_F12_TataNld]
hv_12_Internode = [metric.do(_F12_Internode) for _F12_Internode in hist_F12_Internode]
hv_12_Missouri = [metric.do(_F12_Missouri) for _F12_Missouri in hist_F12_Missouri]
hv_12_Ion = [metric.do(_F12_Ion) for _F12_Ion in hist_F12_Ion]
hv_12_Palmetto = [metric.do(_F12_Palmetto) for _F12_Palmetto in hist_F12_Palmetto]

# kgbdmoea
hv_12 = [(hv_Cogent + hv_UsCarrier + hv_HiberniaGlobal + hv_Colt + hv_Funet + hv_Abvt + hv_Intellifiber + hv_TataNld + hv_Internode + hv_Missouri + hv_Ion + hv_Palmetto) / 12 for hv_Cogent, hv_UsCarrier, hv_HiberniaGlobal, hv_Colt, hv_Funet, hv_Abvt, hv_Intellifiber, hv_TataNld, hv_Internode, hv_Missouri, hv_Ion, hv_Palmetto in zip(hv_12_Cogent, hv_12_UsCarrier, hv_12_HiberniaGlobal, hv_12_Colt, hv_12_Funet, hv_12_Abvt, hv_12_Intellifiber, hv_12_TataNld, hv_12_Internode, hv_12_Missouri, hv_12_Ion, hv_12_Palmetto)]

In [ ]:
print('The hypervolumn for Cogent:')
print(hv_12_Cogent)
print('The hypervolumn for UsCarrier:')
print(hv_12_UsCarrier)
print('The hypervolumn for HiberniaGlobal:')
print(hv_12_HiberniaGlobal)
print('The hypervolumn for Colt:')
print(hv_12_Colt)
print('The hypervolumn for Funet:')
print(hv_12_Funet)
print('The hypervolumn for Abvt:')
print(hv_12_Abvt)
print('The hypervolumn for Intellifiber:')
print(hv_12_Intellifiber)
print('The hypervolumn for TataNld:')
print(hv_12_TataNld)
print('The hypervolumn for Internode:')
print(hv_12_Internode)
print('The hypervolumn for Missouri:')
print(hv_12_Missouri)
print('The hypervolumn for Ion:')
print(hv_12_Ion)
print('The hypervolumn for Palmetto:')
print(hv_12_Palmetto)

# Show all hv for each topo
import matplotlib.pyplot as plt

plt.figure(figsize=(7, 5))
plt.plot(list(range(1, len(hv_12_Cogent)+1)), hv_12_Cogent,  color='black', label='Cogent')
plt.plot(list(range(1, len(hv_12_UsCarrier)+1)), hv_12_UsCarrier,  color='blue', label='UsCarrier')
plt.plot(list(range(1, len(hv_12_HiberniaGlobal)+1)), hv_12_HiberniaGlobal,  color='red', label='HiberniaGlobal')
plt.plot(list(range(1, len(hv_12_Colt)+1)), hv_12_Colt,  color='orange', label='Colt')
plt.plot(list(range(1, len(hv_12_Funet)+1)), hv_12_Funet,  color=(0, 0, 1, 0.5), label='Funet')
plt.plot(list(range(1, len(hv_12_Abvt)+1)), hv_12_Abvt,  color='gray', label='Abvt')
plt.plot(list(range(1, len(hv_12_Intellifiber)+1)), hv_12_Intellifiber,  color='purple', label='Intellifiber')
plt.plot(list(range(1, len(hv_12_TataNld)+1)), hv_12_TataNld,  color='green', label='TataNld')
plt.plot(list(range(1, len(hv_12_Internode)+1)), hv_12_Internode,  color='#33FF57', label='Internode')
plt.plot(list(range(1, len(hv_12_Missouri)+1)), hv_12_Missouri,  color='brown', label='Missouri')
plt.plot(list(range(1, len(hv_12_Ion)+1)), hv_12_Ion,  color='pink', label='Ion')
plt.plot(list(range(1, len(hv_12_Palmetto)+1)), hv_12_Palmetto,  color='olive', label='Palmetto')
plt.title("Convergence")
plt.xlabel("Topos")
plt.ylabel("Hypervolume")
plt.legend()
plt.show()

In [ ]:
print('The aveage of hypervolumn for kgbdmoea:')
print(hv_12)

3. Draw mean hypervolume for kgbdmoea

In [ ]:
import matplotlib.pyplot as plt

#  Show all alogrithms' hv into a single figure
plt.figure(figsize=(7, 5))
# plt.plot(list(range(1, len(hv_1)+1)), hv_1,  color='black', label='AGEMOEA')
# plt.plot(list(range(1, len(hv_2)+1)), hv_2,  color='blue', label='AGEMOEA2')
# plt.plot(list(range(1, len(hv_3)+1)), hv_3,  color='red', label='NSGA2')
# plt.plot(list(range(1, len(hv_4)+1)), hv_4,  color='orange', label='NSGA3')
# plt.plot(list(range(1, len(hv_5)+1)), hv_5,  color=(0, 0, 1, 0.5), label='RNSGA2')
# plt.plot(list(range(1, len(hv_6)+1)), hv_6,  color='gray', label='RNSGA3')
# plt.plot(list(range(1, len(hv_7)+1)), hv_7,  color='purple', label='RVEA')
# plt.plot(list(range(1, len(hv_8)+1)), hv_8,  color='green', label='SMSEMOA')
# plt.plot(list(range(1, len(hv_9)+1)), hv_9,  color='#33FF57', label='UNSGA3')
# plt.plot(list(range(1, len(hv_10)+1)), hv_10,  color='brown', label='CTAEA')
# plt.plot(list(range(1, len(hv_11)+1)), hv_11,  color='pink', label='DNSGA2')
plt.plot(list(range(1, len(hv_12)+1)), hv_12,  color='olive', label='KGBDMOEA')
# plt.plot(list(range(1, len(hv_13)+1)), hv_13,  color='cyan', label='MOEAD')
plt.title("Convergence")
plt.xlabel("Generations") # brown,pink,#5733FF,olive,cyan,#FF5733,#33FF57
plt.ylabel("Hypervolume")
plt.legend()
plt.show()